In [1]:
!pip install graphviz
!pip install pydotplus

You should consider upgrading via the 'c:\users\boomt\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'c:\users\boomt\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [8]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, silhouette_score
from joblib import dump, load
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn import metrics

In [49]:
columnNames = ["timestamp","accX","accY","accZ","gyroX","gyroY","gyroZ","pitch","yaw","roll","answer"]
df_each = [pd.read_csv(filename, header=None, delimiter=',', names=[i for i in columnNames]) for filename in glob.glob("./fall-data-train/*.csv")]
df = pd.concat(df_each, axis=0)

In [50]:
df.head()

,timestamp,accX,accY,accZ,gyroX,gyroY,gyroZ,pitch,yaw,roll,answer
0,1.637594e+09,0.076492,-0.028275,-1.025864,0.083753,0.020969,-0.069271,0.084725,-0.004381,0.083138,0
1,1.637594e+09,0.069962,-0.116333,-1.000702,0.039438,0.027808,0.023531,0.096635,-0.023273,0.078887,0
2,1.637594e+09,0.066223,-0.074051,-0.969040,-0.024988,0.029181,-0.024695,0.137557,-0.036107,0.099456,0
3,1.637594e+09,0.096603,-0.146362,-0.982025,-0.010441,0.003999,0.007406,0.150668,-0.032262,0.097191,0
4,1.637594e+09,0.100342,-0.147705,-0.982132,-0.012069,-0.001667,0.007258,0.149083,-0.031601,0.101344,0


In [51]:
df.shape

(252, 11)

In [52]:
df.describe()

,timestamp,accX,accY,accZ,gyroX,gyroY,gyroZ,pitch,yaw,roll,answer
count,2.520000e+02,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000,252.000000
mean,1.637595e+09,-0.191790,-0.305353,-0.540571,-0.019153,-0.031517,-0.039499,0.435868,-0.699865,-0.401269,0.083333
std,5.622755e+02,0.517224,0.467697,0.584489,0.058203,0.350327,0.593348,0.660127,1.234571,0.655257,0.276935
min,1.637594e+09,-0.949554,-1.000778,-1.025864,-0.442986,-3.141425,-8.874131,-0.926541,-3.035300,-2.513559,0.000000
25%,1.637595e+09,-0.752258,-0.975128,-0.976406,-0.015032,-0.006187,0.001857,-0.000898,-1.962455,-0.990645,0.000000
50%,1.637595e+09,-0.058067,-0.038956,-0.556519,-0.011404,-0.000638,0.005938,0.039305,-0.031947,-0.274583,0.000000
75%,1.637595e+09,0.077793,-0.000008,-0.187496,-0.008210,0.003609,0.008978,1.370682,0.066295,0.097757,0.000000
max,1.637595e+09,4.754272,0.859344,6.398071,0.202296,2.675242,0.692268,1.523645,3.117436,2.716222,1.000000


In [53]:

df_train = df
df_train.drop('timestamp',axis='columns', inplace=True)
# df_train.drop('answer',axis='columns', inplace=True)
# df_train.drop('gyroX',axis='columns', inplace=True)
# df_train.drop('gyroY',axis='columns', inplace=True)
# df_train.drop('gyroZ',axis='columns', inplace=True)


In [54]:
df_train.head()

,accX,accY,accZ,gyroX,gyroY,gyroZ,pitch,yaw,roll,answer
0,0.076492,-0.028275,-1.025864,0.083753,0.020969,-0.069271,0.084725,-0.004381,0.083138,0
1,0.069962,-0.116333,-1.000702,0.039438,0.027808,0.023531,0.096635,-0.023273,0.078887,0
2,0.066223,-0.074051,-0.969040,-0.024988,0.029181,-0.024695,0.137557,-0.036107,0.099456,0
3,0.096603,-0.146362,-0.982025,-0.010441,0.003999,0.007406,0.150668,-0.032262,0.097191,0
4,0.100342,-0.147705,-0.982132,-0.012069,-0.001667,0.007258,0.149083,-0.031601,0.101344,0


In [55]:
df_train.groupby(['answer']).mean()

,accX,accY,accZ,gyroX,gyroY,gyroZ,pitch,yaw,roll
answer,,,,,,,,,
0,-0.222923,-0.331756,-0.552451,-0.014246,-0.020274,0.008291,0.473943,-0.770182,-0.442080
1,0.150672,-0.014919,-0.409892,-0.073133,-0.155192,-0.565192,0.017042,0.073623,0.047652


# Feature Selection

In [56]:
feature_cols = ['accX', 'accY', 'accZ', 'gyroX', 'gyroY', 'gyroZ', 'pitch','yaw','roll']
X = df_train[feature_cols] # Features
Y = df_train.answer # Target variable

In [43]:
X.head()

,accX,accY,accZ,gyroX,gyroY,gyroZ,pitch,yaw,roll
0,0.076492,-0.028275,-1.025864,0.083753,0.020969,-0.069271,0.084725,-0.004381,0.083138
1,0.069962,-0.116333,-1.000702,0.039438,0.027808,0.023531,0.096635,-0.023273,0.078887
2,0.066223,-0.074051,-0.969040,-0.024988,0.029181,-0.024695,0.137557,-0.036107,0.099456
3,0.096603,-0.146362,-0.982025,-0.010441,0.003999,0.007406,0.150668,-0.032262,0.097191
4,0.100342,-0.147705,-0.982132,-0.012069,-0.001667,0.007258,0.149083,-0.031601,0.101344


In [44]:
X.mean()

accX    -0.191790
accY    -0.305353
accZ    -0.540571
gyroX   -0.019153
gyroY   -0.031517
gyroZ   -0.039499
pitch    0.435868
yaw     -0.699865
roll    -0.401269
dtype: float64

In [57]:
Y.head()
print(type(Y))

<class 'pandas.core.series.Series'>


In [62]:
from river import compose
from river import datasets
from river import metrics
from river import preprocessing
from river import anomaly

model = compose.Pipeline(
         preprocessing.MinMaxScaler(),
         anomaly.HalfSpaceTrees(seed=42)
         )

auc = metrics.ROCAUC()
labels = Y.values.tolist()

for index, x in enumerate(X.iterrows()):
    data = {}
    for y in range(x[1].size):
        data[y] = x[1][y]

    # print(data)
    score = model.score_one(data)
    print(score)
    model = model.learn_one(data, labels[index])
    auc = auc.update(labels[index], score)

# for x, y in datasets.CreditCard().take(8000):
#     print(x)
#     score = model.score_one(x)
#     model = model.learn_one(x, y)
#     auc = auc.update(y, score)

auc

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0.748986301369863
0.9706692759295499
0.974185518590998


ROCAUC: 0.545661

# Dump Model

In [60]:
dump(model, '../Patient/fall_model.h5')

['../Patient/fall_model.h5']

In [76]:
dump(dt_scaler, '../SpaceWars/model/scaler.bin')

['../SpaceWars/model/scaler.bin']

In [89]:
from river import cluster
from river import stream
from river import tree
from river import evaluate
from river import metrics
import matplotlib.pyplot as plt
import joblib


In [107]:
df_train.head()

,accX,accY,accZ,pitch,yaw,roll,answer
0,-0.034027,-0.972977,-0.240845,1.393136,0.311162,-0.356315,0
1,-0.032669,-0.995636,-0.138977,1.397252,0.108693,-0.142706,0
2,0.008255,-0.977600,-0.208328,1.362467,-0.072950,0.038964,0
3,-0.080521,-0.966339,-0.223251,1.330597,0.203506,-0.327716,0
4,-0.069138,-0.968109,-0.195251,1.358875,0.180678,-0.302742,0


In [108]:

model = tree.HoeffdingTreeClassifier(grace_period=50)
train_datas = X.values.tolist()
labels = y.values.tolist()

for index, x in enumerate(X.iterrows()):
    data = {}
    for y in range(x[1].size):
        data[y] = x[1][y]
    
    print(f'{data}------> {labels[index]}')
    model.learn_one(data, labels[index])

# for index, x in enumerate(train_datas):
#     # print(f'X = {x}, y = {labels[index]}')
#     model.learn_one(x, labels[index])

model

{0: -0.034027099609375, 1: -0.9729766845703124, 2: -0.2408447265625, 3: 1.3931363779142474, 4: 0.3111622241905467, 5: -0.3563154519913828}------> 0
{0: -0.0326690673828125, 1: -0.995635986328125, 2: -0.13897705078125, 3: 1.3972515638073857, 4: 0.1086932966016528, 5: -0.1427064567667436}------> 0
{0: 0.0082550048828125, 1: -0.97760009765625, 2: -0.2083282470703125, 3: 1.3624666100335023, 4: -0.0729498866835961, 5: 0.0389642457494368}------> 0
{0: -0.0805206298828125, 1: -0.966339111328125, 2: -0.2232513427734375, 3: 1.3305974638195497, 4: 0.2035063390392016, 5: -0.327715663830461}------> 0
{0: -0.0691375732421875, 1: -0.968109130859375, 2: -0.19525146484375, 3: 1.358875130312022, 4: 0.1806783133276029, 5: -0.3027421252085616}------> 0
{0: -0.0585174560546875, 1: -0.9629974365234376, 2: -0.207244873046875, 3: 1.3488407281269517, 4: 0.1842601835484717, 5: -0.2727022211172044}------> 0
{0: -0.0643768310546875, 1: -0.97216796875, 2: -0.20806884765625, 3: 1.3450320719483018, 4: 0.18901098212

HoeffdingTreeClassifier (
  grace_period=50
  max_depth=inf
  split_criterion="info_gain"
  split_confidence=1e-07
  tie_threshold=0.05
  leaf_prediction="nba"
  nb_threshold=0
  nominal_attributes=None
  splitter=GaussianSplitter (
    n_splits=10
  )
  binary_split=False
  max_size=100
  memory_estimate_period=1000000
  stop_mem_management=False
  remove_poor_attrs=False
  merit_preprune=True
)

In [109]:
test_data = {0: -0.034027099609375, 1: -0.9729766845703124, 2: -0.2408447265625, 3: 1.3931363779142474, 4: 0.3111622241905467, 5: -0.3563154519913828}
y_pred = model.predict_one(test_data)
print(y_pred)

0


In [91]:
joblib.dump(model, 'model.h5')

['model.h5']